<a href="https://colab.research.google.com/github/Andrei198203/Data_Science_HW_7/blob/main/HW7_Data_Science.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install scikit-surprise

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 772.0/772.0 kB 7.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.3-cp310-cp310-linux_x86_64.whl size=3162993 sha256=29b37d6556c65bff5cd440a4604c6fde337fad976613f7c9302e3cde8783ab02
  Stored in directory: /root/.cache/pip/wheels/a5/ca/a8/4e28def53797fdc4363ca4af740db15a9c2f1595ebc51fb445
Successfully built scikit-surprise


In [3]:
from surprise import SVD
from surprise import Dataset
from surprise.model_selection import cross_validate
import pandas as pd
import glob

In [6]:
# Load the movielens-100k dataset (download it if needed),
data = Dataset.load_builtin('ml-100k', prompt=False)

In [7]:
# Or if you'd rather, comment the above line and uncomment the below line
# for a larger data set, while anticipating longer training times.

# data = Dataset.load_builtin(name='ml-1M', prompt=False)

# We'll use the famous SVD algorithm.
algo = SVD()

In [8]:
# Run 5-fold cross-validation and print results
cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9374  0.9357  0.9448  0.9308  0.9416  0.9381  0.0048  
MAE (testset)     0.7396  0.7364  0.7445  0.7329  0.7431  0.7393  0.0043  
Fit time          1.40    1.38    1.99    1.92    1.45    1.63    0.27    
Test time         0.13    0.20    0.20    0.25    0.12    0.18    0.05    


{'test_rmse': array([0.93735793, 0.93566936, 0.94479689, 0.93083364, 0.94162085]),
 'test_mae': array([0.73959221, 0.7363766 , 0.74448965, 0.73289406, 0.7430643 ]),
 'fit_time': (1.3957712650299072,
  1.377030849456787,
  1.9898419380187988,
  1.915909767150879,
  1.4539196491241455),
 'test_time': (0.13356232643127441,
  0.20455050468444824,
  0.19907140731811523,
  0.24722504615783691,
  0.11749696731567383)}

Splitting up the dataset

In [9]:
from surprise import accuracy
from sklearn.metrics import mean_absolute_error
from surprise.model_selection import train_test_split

In [10]:
# sample random trainset and testset
# test set is made of 25% of the ratings.
trainset, testset = train_test_split(data, test_size=.25)

In [11]:
# We'll use the famous SVD algorithm.
algo = SVD()

In [12]:
# Train the algorithm on the trainset, and predict ratings for the testset
algo.fit(trainset)
predictions = algo.test(testset)

In [13]:
# Then compute RMSE
accuracy.rmse(predictions)

RMSE: 0.9340


0.9339643899409129

Set a baseline with a simple grid search

In [14]:
from surprise.model_selection import GridSearchCV
param_grid = {'n_epochs': [5, 10], 'lr_all': [0.002, 0.005],
              'reg_all': [0.4, 0.6]}
gs = GridSearchCV(SVD, param_grid, measures=['rmse', 'mae'], cv=3)

gs.fit(data)

In [15]:
# best RMSE score
print(gs.best_score['rmse'])

0.9638565988851501


In [16]:
# combination of parameters that gave the best RMSE score
print(gs.best_params['rmse'])

{'n_epochs': 10, 'lr_all': 0.005, 'reg_all': 0.4}


In [17]:
!pip install sigopt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 198.8/198.8 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.1/58.1 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.8/143.8 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 6.6 MB/s eta 0:00:00
  Attempting uninstall: urllib3
    Found existing installation: urllib3 2.0.7
    Uninstalling urllib3-2.0.7:
      Successfully uninstalled urllib3-2.0.7


In [20]:
# Pass your API token directly, overriding any environment variables
from sigopt import Connection
conn = Connection(client_token="YOUR_API_TOKEN_HERE")

In [24]:
experiment = conn.experiments().create(
  name='Surprise Movies-100k Recommender',
  # Define which parameters you would like to tune
  parameters=[
    dict(name='n_epochs', type='int', bounds=dict(min=5, max=10)),
    dict(name='lr_all', type='double', bounds=dict(min=0.002, max=0.005)),
    dict(name='reg_all', type='double', bounds=dict(min=0.4, max=0.6))
  ],
  metrics=[
    dict(name='RMSE', objective='minimize', strategy='optimize'),
    dict(name='MAE', objective='minimize', strategy='store')
  ],
  parallel_bandwidth=1,
  # Define an Observation Budget for your experiment
  observation_budget=50,
)
print("Created experiment: https://app.sigopt.com/experiment/" + experiment.id)

ConnectionException: ConnectionException: An error occurred connecting to SigOpt.
Contact support@sigopt.com for assistance.

ExpiringHTTPSConnectionPool(host='api.sigopt.com', port=443): Max retries exceeded with url: /v1/experiments (Caused by NewConnectionError('<sigopt.urllib3_patch.SigOptHTTPSConnection object at 0x7a17c8c11ae0>: Failed to establish a new connection: [Errno -2] Name or service not known'))

In [22]:
# Evaluate your model with the suggested parameter assignments
def create_model(assignments):
  algo = SVD(
      n_epochs=assignments['n_epochs'],
      lr_all=assignments['lr_all'],
      reg_all=assignments['reg_all']
  ).fit(trainset)
  return algo

def evaluate_model(assignments):
  algo = create_model(assignments)
  predictions = algo.test(testset)
  # Then compute RMSE and MAE:
  return [
      dict(name="RMSE", value=accuracy.rmse(predictions)),
      dict(name="MAE", value=accuracy.mae(predictions))
    ]

In [23]:
# Run the Optimization Loop until the Observation Budget is exhausted
while experiment.progress.observation_count < experiment.observation_budget:
  suggestion = conn.experiments(experiment.id).suggestions().create()
  value_dicts = evaluate_model(suggestion.assignments)
  conn.experiments(experiment.id).observations().create(
    suggestion=suggestion.id,
    values=value_dicts,
  )

NameError: name 'experiment' is not defined

Alternative with scikit-optimize

In [25]:
!pip install scikit-optimize

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.7/107.7 kB 2.4 MB/s eta 0:00:00


In [26]:
from surprise import Dataset
from surprise.model_selection import cross_validate
from surprise import SVD
from skopt import forest_minimize

In [27]:
# Movielens dataset download
data = Dataset.load_builtin('ml-100k', prompt=False)

In [32]:
# Definition of the function to be optimized
def objective(params):
    # Unpacking hyperparameters
    n_epochs, lr_all = params

    # Create an SVD model with current hyperparameters
    algo = SVD(n_epochs=int(n_epochs), lr_all=lr_all)

    # Running cross-validation with new hyperparameters and returning the average RMSE
    results = cross_validate(algo, data, measures=['RMSE'], cv=3, verbose=False)
    return results['test_rmse'].mean()

In [33]:
# Defining the search space for hyperparameters
space = [(5, 20),              # n_epochs
         (0.002, 0.005)]       # lr_all

In [34]:
# Start optimizing hyperparameters
result = forest_minimize(objective, space, n_calls=20, random_state=42)

In [35]:
# Output of optimization results
print("Best RMSE found:", result.fun)
print("Best parameters found:", result.x)

Best RMSE found: 0.9456918190446674
Best parameters found: [20, 0.004976634677873654]


Alternative with Hyperopt

In [36]:
!pip install hyperopt

In [37]:
from hyperopt import fmin, tpe, hp, Trials
from surprise import Dataset
from surprise.model_selection import cross_validate
from surprise import SVD

In [38]:
# Movielens dataset download
data = Dataset.load_builtin('ml-100k', prompt=False)

In [39]:
# Означення функції об'єктиву
def objective(params):
    # Створення моделі SVD з параметрами
    algo = SVD(n_epochs=int(params['n_epochs']), lr_all=params['lr_all'])

    # Проведення крос-валідації з новими гіперпараметрами та повернення середнього RMSE
    results = cross_validate(algo, data, measures=['RMSE'], cv=3, verbose=False)
    return results['test_rmse'].mean()

In [40]:
# Визначення простору пошуку гіперпараметрів
space = {
    'n_epochs': hp.quniform('n_epochs', 5, 20, 1),       # кількість епох
    'lr_all': hp.uniform('lr_all', 0.002, 0.005)         # швидкість навчання
}

In [41]:
# Запуск оптимізації гіперпараметрів
trials = Trials()
best = fmin(fn=objective, space=space, algo=tpe.suggest, max_evals=20, trials=trials)

100%|██████████| 20/20 [01:20<00:00,  4.05s/trial, best loss: 0.9491671499691908]


In [42]:
# Виведення результатів оптимізації
print("Best RMSE found:", trials.best_trial['result']['loss'])
print("Best parameters found:", best)

Best RMSE found: 0.9491671499691908
Best parameters found: {'lr_all': 0.004902699905827243, 'n_epochs': 13.0}


Alternative with  SVD++, NMF

In [43]:
from surprise import Dataset, Reader
from surprise.model_selection import GridSearchCV
from surprise import SVD, SVDpp, NMF

In [44]:
# Завантаження датасету Movielens
data = Dataset.load_builtin('ml-100k')


In [45]:
# Означення алгоритмів, які ми хочемо перевірити
algos = [
    {'name': 'SVD', 'algo_class': SVD},
    {'name': 'SVD++', 'algo_class': SVDpp},
    {'name': 'NMF', 'algo_class': NMF}
]

In [46]:
# Означення параметрів, які ми хочемо перевірити для кожного алгоритму
param_grid = {
    'SVD': {'n_epochs': [5, 10, 20], 'lr_all': [0.002, 0.005]},
    'SVD++': {'n_epochs': [5, 10, 20], 'lr_all': [0.002, 0.005]},
    'NMF': {'n_epochs': [5, 10, 20]}
}

In [48]:
# Знаходження оптимальних параметрів за допомогою крос-валідації
for algo_info in algos:
    algo_name = algo_info['name']
    algo_class = algo_info['algo_class']
    param_grid_algo = param_grid[algo_name]
    gs = GridSearchCV(algo_class, param_grid_algo, measures=['rmse', 'mae'], cv=3)
    gs.fit(data)
    print(f"Best RMSE for {algo_name}: {gs.best_score['rmse']}")

Best RMSE for SVD: 0.943921286162992
Best RMSE for SVD++: 0.9269378025101105
Best RMSE for NMF: 0.9848705830353343
